In [8]:
import os
import time 
import numpy as np
from __future__ import print_function

import tensorflow as tf

In [135]:
class N2N_LSTM:
    def __init__(self,
                 seq_max_len=10,  
                 input_len=40,
                 output_len=40,
                 batch_size=1,
                 lstm_size=[40, 80, 80, 80, 40],
                 num_layers=5,
                 learning_rate=0.001,
                 grad_clip=5,
                 keep_prob=1.,
                 ):
            
        self.seq_max_len = seq_max_len
        self.input_len = input_len
        self.output_len = output_len
        self.batch_size = batch_size
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.learning_rate = learning_rate
        self.grad_clip = grad_clip
        self.train_keep_prob = keep_prob
        
        tf.reset_default_graph() #Clears the default graph stack and resets the global default graph
        self.build_inputs()
        self.build_lstm()
        self.build_loss()
        self.build_optimizer()
        self.saver = tf.train.Saver() #Saves and restores variables.
        
    
    def build_inputs(self):
        with tf.name_scope('inputs'):
            self.inputs = tf.placeholder(tf.float32, 
                                         shape=(self.batch_size, self.seq_max_len, self.input_len),
                                         name='inputs')
            self.targets = tf.placeholder(tf.float32,
                                          shape=(self.batch_size, self.seq_max_len, self.output_len),
                                          name='targets')
            self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
            
            
            
    def build_lstm(self):
        def get_a_cell(lstm_size, keep_prop):
            lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
            drop = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=self.train_keep_prob)
            return drop
        
        with tf.name_scope('lstm'):
            cell = tf.nn.rnn_cell.MultiRNNCell(
                         [get_a_cell(size, self.keep_prob) for size in self.lstm_size]
                                              )
            self.initial_state = cell.zero_state(self.batch_size, tf.float32)
            
            # 透過dynamic_rnn對cell展開時間維度
            self.lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cell, 
                                                                    self.inputs, 
                                                                    initial_state=self.initial_state)
            
            # self.sigmoid_outputs = tf.nn.sigmoid(self.lstm_outputs, name='sigmoid')
            
            '''
            # 透過lstm_outputs得到機率
            seq_output = tf.concat(self.lstm_outputs, 1)
            x = tf.reshape(seq_output, [-1, self.lstm_size[-1]])

            with tf.variable_scope('sigmoid'):
                sigmoid_w = tf.Variable(tf.truncated_normal([self.lstm_size[-1],self.output_len], stddev=0.1))
                sigmoid_b = tf.Variable(tf.zeros(self.output_len))

            self.logits = tf.matmul(x, sigmoid_w) + sigmoid_b
            self.proba_prediction = tf.nn.sigmoid(self.logits, name='predictions')
            '''
            
            
    def build_loss(self):
        with tf.name_scope('loss'):
            y_reshaped = tf.reshape(self.targets, self.lstm_outputs.get_shape())
            loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.lstm_outputs, labels=y_reshaped)
            self.loss = tf.reduce_mean(loss)
            
            

    def build_optimizer(self):
        # 使用clipping gradients
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), self.grad_clip)
        train_op = tf.train.AdamOptimizer(self.learning_rate)
        self.optimizer = train_op.apply_gradients(zip(grads, tvars))



            

    def train(self, batch_generator, iters=10, save_path='./models', save_every_n=200, log_every_n=50):
        self.session = tf.Session()
        with self.session as sess:
            sess.run(tf.global_variables_initializer())
            # Train network
            
            new_state = sess.run(self.initial_state)
            for ite in range(iters):
                step = 0
                print('iters',ite)
                for x, y in batch_generator:
                    step += 1
                    start = time.time()
                    feed = {self.inputs: x,
                            self.targets: y,
                            self.keep_prob: self.train_keep_prob,
                            self.initial_state: new_state}
                    batch_loss, new_state, _ = sess.run([self.loss,
                                                         self.final_state,
                                                         self.optimizer],
                                                        feed_dict=feed)

                    end = time.time()

                    # control the print lines
                    if step % log_every_n == 0:
                        print('step: {} in iter: {}/{}... '.format(step, ite, iters),
                              'loss: {:.4f}... '.format(batch_loss),
                              '{:.4f} sec/batch'.format((end - start)))
                    if (step % save_every_n == 0):
                        self.saver.save(sess, os.path.join(save_path, 'model'), global_step=step)

In [136]:
def main():
    model_path = './models'
    if os.path.exists(model_path) is False:
        os.makedirs(model_path)
    g = batch_generator()
    model = N2N_LSTM()
    model.train(g,
                iters=10,
                save_path=model_path,
                save_every_n=200,
                log_every_n =10
                )


if __name__ == '__main__':
    main()


iters 0
step: 10 in iter: 0/10...  loss: 0.6933...  0.0157 sec/batch
step: 20 in iter: 0/10...  loss: 0.6929...  0.0151 sec/batch
step: 30 in iter: 0/10...  loss: 0.6930...  0.0236 sec/batch
step: 40 in iter: 0/10...  loss: 0.6926...  0.0158 sec/batch
step: 50 in iter: 0/10...  loss: 0.6941...  0.0154 sec/batch
step: 60 in iter: 0/10...  loss: 0.6940...  0.0160 sec/batch
step: 70 in iter: 0/10...  loss: 0.6943...  0.0155 sec/batch
step: 80 in iter: 0/10...  loss: 0.6931...  0.0174 sec/batch
step: 90 in iter: 0/10...  loss: 0.6930...  0.0152 sec/batch
step: 100 in iter: 0/10...  loss: 0.6928...  0.0148 sec/batch
step: 110 in iter: 0/10...  loss: 0.6935...  0.0147 sec/batch
step: 120 in iter: 0/10...  loss: 0.6927...  0.0149 sec/batch
step: 130 in iter: 0/10...  loss: 0.6926...  0.0146 sec/batch
step: 140 in iter: 0/10...  loss: 0.6924...  0.0157 sec/batch
step: 150 in iter: 0/10...  loss: 0.6930...  0.0202 sec/batch
step: 160 in iter: 0/10...  loss: 0.6924...  0.0153 sec/batch
step: 170

In [102]:
# return: batch_generator
#g = batch_generator(arr, FLAGS.num_seqs, FLAGS.num_steps)

def batch_generator(input_len=40, output_len=40, batch_size=1, total=3000, seq_len=10):
    runs = int(total/seq_len/batch_size)
    
    for run in range(runs):
        x = np.zeros(shape=(batch_size, seq_len, input_len))
        y = np.zeros(shape=(batch_size, seq_len, output_len))
        for seq in range(seq_len):
            for batch in range(batch_size):
                sequen = np.random.randint(2, size=input_len)
                x[batch, seq, :] = sequen
                y[batch, seq, :] = np.abs(sequen - 1)
        yield (x, y)
